# RAG(검색-증강 생성) Retrieval-Augmented Generation
- LLM이 외부 데이터를 컨텍스트로 활용.

In [ ]:
# import os
# os.environ.update({
#     "OMP_NUM_THREADS": "1",
#     "OPENBLAS_NUM_THREADS": "1",
#     "MKL_NUM_THREADS": "1",
#     "VECLIB_MAXIMUM_THREADS": "1",
#     "NUMEXPR_NUM_THREADS": "1",
# })

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:
%pip install -q "langchain_chroma>=0.1.2" langchain_community faiss-cpu langchain

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS

# OpenAI 임베딩 모델
embedding = OpenAIEmbeddings()

# x = embedding.embed_query('오늘 점심은 배불리 먹어서 졸려')
# print(len(x))

# VectorStore 에 임베딩 후 저장(In memory)
vectorstore = FAISS.from_documents(
    documents,
    embedding=embedding
)

In [16]:
vectorstore.similarity_search(
    '초보자가 키우기 좋은 애완동물 추천해줘', k=2
)

[Document(id='52092506-f834-44d1-8a3d-565aece957d2', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 12}, page_content='∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 \n단계적으로 탑재할 계획\nn 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 \n이미지 모델의 3개 모델로 구성\n∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 \n처리를 지원\n∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 \n사내 소프트웨어 개발에 최적화\n∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 \n저해상도 이미지의 고해상도 전환도 지원'),
 Document(id='7d071ec4

## 사전처리 단계
1. 문서 불러오기 (Loading)
2. 텍스트 나누기 (Splitting)
3. 숫자로 바꾸기 (Embedding)
4. 저장하기 (VectorStore)

In [9]:
%pip install -q pymupdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# 1. Load
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('./data/spri.pdf')
docs = loader.load()

print('원본 pdf의 장 수', len(docs))

23


In [ ]:
# 2. Split
from langchain_text_splitters import RecursiveCharacterTextSplitter
# 500글자당 1 청크 / 50글자는 겹치게 나눈다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

# for doc in split_docs:
#     print('----')

print('분할 후 청크 수', len(split_docs))

분할 후 청크 수 72


In [ ]:
# 3. 임베딩, 4. 벡터스토어 저장
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_docs, embedding=embedding)

# Test
vectorstore.similarity_search(
    '미국 대통령과 관련된 문서들 가져와봐', k=4
)

[Document(id='000772ab-01e8-438f-a3ef-2f3ca9656e0e', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 3}, page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표 \nn 미국 바이든 대통령이 ‘안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령’에 서명하고 \n광범위한 행정 조치를 명시\nn 행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 \n보호 △노동자 지원 △혁신과 경쟁 촉진 △국제협력을 골자로 함\nKEY Contents\n£ 바이든 대통령, AI 행정명령 통해 안전하고 신뢰할 수 있는 AI 개발과 활용 추진\nn 미국 바이든 대통령이 2023년 10월 30일 연방정부 차원에서 안전하고 신뢰할 수 있는 AI 개발과 \n사용을 보장하기 위한 행정명령을 발표\n∙행정명령은 △AI의 안전과 보안 기준 마련 △개인정보보호 △형평성과 시민권 향상 △소비자 보호 \n△노동자 지원 △혁신과 경쟁 촉진 △국제협력에 관한 내용을 포괄'),
 Docu

## 검색증강 단계
1. 사용자 질문 (Query)
2. 검색 (Retrieve)
3. LLM
4. 최종 답변

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain import hub

# Prompt 세팅
prompt = hub.pull('rlm/rag-prompt')

# LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기 생성(retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('삼성전자 관련 소식 다 가져와')

'삼성전자는 자체 개발 생성 AI ‘삼성 가우스’를 공개했으며, 언어, 코드, 이미지의 3개 모델로 구성되어 온디바이스에서 작동 가능합니다. 이 AI는 메일 작성, 문서 요약, 번역, AI 코딩 어시스턴트와 이미지 생성 등 다양한 기능을 지원하며, 단계적으로 제품에 탑재될 예정입니다. 추가로, 2023년 11월 8일 열린 ‘삼성 AI 포럼’에서 처음 공개되었으며, 경쟁 제품들과의 차별성을 강조하고 있습니다.'

In [25]:
# 관심있는 데이터소스로 RAG

loader = PyMuPDFLoader('./data/SQLP.pdf')
sqlp_doc = loader.load()

print('원본 pdf의 장 수', len(sqlp_doc))

원본 pdf의 장 수 445


In [31]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_sqlp = text_splitter.split_documents(sqlp_doc)

print('분할 후 청크 수', len(split_sqlp))

분할 후 청크 수 548


In [32]:
embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_sqlp, embedding=embedding)

# Test
vectorstore.similarity_search(
    '정규화 관련 개념들 가져와봐', k=10
)

[Document(id='6a9b1d8b-0a8e-4665-ad94-eb5ea9075a3c', metadata={'producer': 'macOS 버전 15.5(빌드 24F74) Quartz PDFContext', 'creator': '', 'creationdate': "D:20250806012743Z00'00'", 'source': './data/SQLP.pdf', 'file_path': './data/SQLP.pdf', 'total_pages': 445, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20250806012743Z00'00'", 'trapped': '', 'modDate': "D:20250806012743Z00'00'", 'creationDate': "D:20250806012743Z00'00'", 'page': 81}, page_content='제장데이터모델과성능\n2\n데이터자격검정\nKdata\n- 79 -\nwww.dataq.or.kr\n제절반정규화와성능\n3\n반정규화를통한성능향상전략\n1.\n가\n반정규화의정의\n.\n반정규화\n역정규화\n용어는조금다르게표현되어도그의미는동일하다\n여기에서반\n(=\n)\n.\n정규화는\n반\n의의미가아닌한자로반대하다의의미를가진\n의의미이다\n영어로\n‘\n(Half)’\n‘\n’\n.\n反\n는\n이다\n비정규화는아예정규화를수행하지않은모델을지칭할때사\nDe-Normalization\n.\n용한다.\n반정규화를정의하면정규화된엔터티\n속성\n관계에대해시스템의성능향상과개발\n,\n,\n과운영\n의단순화를위해중복\n통합\n분리등을수행하는데\n(Development)\n(Maintenance)\n,\n,\n이터모델링의기법을의미한다\n협의의반정규화는데이터를중복하여성능을향상시키기\n.\n위한기법이라고정의할수있고좀더넓은의미의반정규화는성능을향상시키기위해정\n규화된데이터모델에서중복통합\n분리등을수행하

In [33]:
# Prompt 세팅
prompt = hub.pull('rlm/rag-prompt')

# LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기 생성(retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke('제2정규화에 대한 개념 설명해줘')

'제2정규화는 데이터베이스 설계에서 부분 함수 종속을 제거하여 데이터 중복을 최소화하는 과정입니다. 이를 통해 데이터 무결성을 높이고 저장 공간을 효율적으로 사용할 수 있습니다. 그러나 성능 향상이나 목표에 따라 반정규화와 함께 고려하기도 합니다.'

In [34]:
# Agent + RAG
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_tavily import TavilySearch
from datetime import datetime
# RAG 관련
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

today = datetime.today().strftime('%Y-%m-%d')

llm = ChatOpenAI(model='gpt-4.1-nano')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

loader = PyMuPDFLoader('./data/spri.pdf')
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embedding=embedding)
retriever = vectorstore.as_retriever()

rag_tool = create_retriever_tool(
    retriever,
    name='pdf_search',
    description='PDF 문서에서 질문과 관련된 내용을 검색합니다.'  # Agent가 언제 이 tool을 쓸지 알게됨
)

text = f"""
너는 웹 검색이 가능하고, 2023년 12월 인공지능 산업 최신동향 정보를 담은 pdf 를 검색할 수 있는 어시스턴트야.

- 사용자가 PDF 문서와 관련된 질문(ex. '이 pdf에서', '문서내용', '파일에서)을 하면 반드시 'pdf_search' 도구를 써야해
- 사용자 질문이 팩트체크를 필요로 하고, 최신성이 필요하다 판단되면 web_search를 실행해야해
- 사용자가 일반적인 질문을 하고, 최신성이나 팩트체크가 필요없으면 그냥 답변해
- 뭔가 확실하지 않으면 pdf_search 와 web_search를 모두 실행해서 답변을 생성해

오늘은 {today} 야.
"""


prompt = ChatPromptTemplate.from_messages([
    ('system', text),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')  # 도구(검색) 호출때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool, rag_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    memory=memory, 
    tools=[search_tool, rag_tool],
    verbose=True)

C:\Users\배성우\AppData\Local\Temp\ipykernel_15824\3050205485.py:56: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [35]:
agent_executor.invoke({'input': '방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?'})



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': '2023년 이후 인공지능 산업 및 산학동향 최신 정보', 'search_depth': 'advanced', 'time_range': 'year'}`


{'query': '2023년 이후 인공지능 산업 및 산학동향 최신 정보', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://ettrends.etri.re.kr/ettrends/212/0905212008/', 'title': '인공지능 분야 산업·기술 동향 및 이슈', 'content': 'D. Pinto, "NVIDIA Announces Project GR00T Foundation Model for Humanoid Robots and Major Isaac Robotics Platform Update," NVIDIA Newsroon, Mar. 18, 2024. \n\nSamsung, "온디바이스AI." \n\n임양섭, "멀티모달 AI 경쟁과 다가오는 AGI," IITPICT SPOT ISSUE ICT 산업동향, SPOT 2024-02, 2024, pp. 1-21.\n\nM.R. Morris et al., "Levels of AGI for Operationalizing Progress on the Path to AGI," Proceedings of ICML, 2024. \n\nForbes, "OpenAI’s 5 Levels of ‘Super AI’(AGI To Outperform Human Capability)," July 16, 2024. [...] Marketsandmarkets, "Artificial Intelligence (AI) Market," May. 2024.\n\nGartner, "Gartner Top 10 Strategic Technology

{'input': '방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?',
 'chat_history': [HumanMessage(content='방금 말한 내용들 2023년 기준인데, 요즘엔 어떻게 됐대?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2023년 이후 인공지능 산업은 빠른 성장과 함께 다양한 발전 동향이 이어지고 있습니다. 최신 자료에 따르면, 글로벌 인공지능 시장 규모는 2023년 기준 2,146억 달러에 달하며, 2030년에는 약 1조 3,391억 달러로 연평균 35.7%의 성장률을 기록할 것으로 예상됩니다. 특히 생성형 인공지능(GPT 등)의 발전이 산업 전반에 큰 영향을 미치고 있으며, 주요 기업들은 대규모 투자를 지속하고 있습니다.\n\n국내 상황도 긍정적입니다. 2024년 국내 인공지능 시장은 약 6조 3,000억 원 규모로 성장하였으며, AI 응용 소프트웨어가 시장의 절반 이상을 차지하는 등 실생활과 산업현장에서 적극 활용되고 있습니다. 다만, 인력 부족이 큰 과제로 남아 있으며, 인재 양성과 글로벌 시장 진출 지원이 중요한 이슈로 부각되고 있습니다.\n\n또한, 각국 정부와 기업들은 AI 기술 생태계 자립화와 인프라 확충, 윤리·책임성 확보를 위한 정책을 추진하고 있으며, 산업적·기술적 혁신이 지속되고 있음을 알 수 있습니다. \n\n더 구체적인 최신 사례와 정책 동향은 관련 자료를 참고하시면 도움이 될 것입니다.', additional_kwargs={}, response_metadata={})],
 'output': '2023년 이후 인공지능 산업은 빠른 성장과 함께 다양한 발전 동향이 이어지고 있습니다. 최신 자료에 따르면, 글로벌 인공지능 시장 규모는 2023년 기준 2,146억 달러에 달하며, 2030년에는 약 1조 3,391억 달러로 연평균 35.7%의 성장률을 기록할 것으로 예상됩니다. 특히 생성형 인공지능(GPT 등)의 발전이 산업 